In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
torch.manual_seed(42)

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
device

'cpu'

In [6]:
df_train = pd.read_csv('fashion-mnist_train.csv')
df_test = pd.read_csv('fashion-mnist_test.csv')

In [7]:
X_train = df_train.iloc[:,1:].values
y_train = df_train.iloc[:,0].values

In [8]:
X_test = df_test.iloc[:,1:].values
y_test = df_test.iloc[:,0].values

In [12]:
#transformations
from torchvision.transforms import transforms

custom_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]
)])
    


In [57]:
from PIL import Image
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, features, labels, transforms):
        self.features = features
        self.labels = labels
        self.transforms = transforms
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):
        #resize to (28,28)
        image = self.features[index].reshape(28,28)
        
        
        #change datatype to np.uint8
        image = image.astype(np.uint8)
        
        #change black&white to color
        image = np.stack([image] * 3, axis=-1)
        
        #convert array to PIL image 
        image = Image.fromarray(image)
        
        
        #apply transform
        image = self.transforms(image)
        
        #return
        return image, torch.tensor(self.labels[index], dtype=torch.long)
        
        
        

In [58]:
train_dataset = CustomDataset(X_train, y_train, transforms = custom_transform)
test_dataset = CustomDataset(X_test, y_test, transforms=custom_transform)

In [59]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True
)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False
)

In [47]:
#fetch the pretrained model

import torchvision.models as models

vgg16 = models.vgg16(pretrained=True)

/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [48]:
vgg16.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [49]:
vgg16.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [50]:
for param in vgg16.features.parameters():
    param.requires_grad=False
    #freeaing the weights of feature extracting part of VGGNET

In [51]:
vgg16.classifier = nn.Sequential(
    nn.Linear(25088,1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024,512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512,10),
    
)

In [52]:
vgg16.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=1024, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=1024, out_features=512, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=512, out_features=10, bias=True)
)

In [53]:
vgg16 = vgg16.to(device)

In [54]:
LEARNING_RATE = 0.0001
EPOCHS = 20


In [55]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.classifier.parameters(), lr=LEARNING_RATE)

In [ ]:
#training loop

for epoch in range(EPOCHS):
    total_epoch_loss = 0
    for batch_features, batch_labels in train_dataloader:
        #forward pass
        outputs = vgg16(batch_features)
        
        
        #loss calculate
        loss = loss_fn(outputs, batch_labels)
        
        #back pass
        optimizer.zero_grad()
        loss.backward()
        
        #update grads
        optimizer.step()
        
        total_epoch_loss = total_epoch_loss + loss.item()
    
    avg_loss = total_epoch_loss / len(train_dataloader)
    print(f"Epoch: {epoch + 1}, Loss: {avg_loss}")
    